# Agent Evaluation for Customer Support: Comparing Models and Parameters

<table align="left">
  <td style="text-align: center">
    <a href="#">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="#">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="#">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="#">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="#" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="#" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="#" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="#" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="#" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| Authors | [Anish Shah](https://github.com/ash0ts) |

## Overview

This notebook demonstrates how to generate synthetic evaluation data and compare different models and parameters for a customer support agent. We'll explore three main types of evaluations:

1. Final Response Evaluation: Assessing the agent's final answer
2. Single Step Evaluation: Evaluating individual tool selections
3. Trajectory Evaluation: Analyzing the complete path of actions

The tutorial uses the following Google Cloud services and resources:
* [Vertex AI Gen AI Evaluation](https://cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-overview)
* [Weights and Biases Weave](https://wandb.me/tryweave)

The steps performed include:
* Generating synthetic evaluation data
* Setting up evaluation metrics
* Comparing model performance
* Analyzing evaluation results

**NOTE:** Please run the `setup.py` script in this folder before running the notebook.

In [1]:
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/ash0ts/generative-ai.git
    %cd generative-ai/gemini/evaluation/synthetic-data-evals
    !pip install -qqq uv 
    !uv pip install --system --requirements pyproject.toml
except:
    pass

In [ ]:
from set_env import set_env
set_env("GEMINI_API_KEY")
# set_env("HUGGING_FACE_HUB_TOKEN")
set_env("VERTEX_PROJECT_ID")
set_env("VERTEX_LOCATION")
set_env("VERTEX_MODEL_ID")
set_env("VERTEX_ENDPOINT_ID")
# set_env("DEEPSEEK_ENDPOINT_ID")
print("Set API Keys")

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from rich.console import Console
from rich.table import Table
import weave
from typing import List, Dict, Any, Tuple
import os

# Import our modules
from evaluator import AgentEvaluator, load_dataset
from dataset_generator import DatasetGenerator, create_customer_support_agent_evaluation_dataset
from customer_support_agent import create_customer_support_agent
from config import WEAVE_PROJECT_NAME

try:
    in_jupyter = True
except ImportError:
    in_jupyter = False
if in_jupyter:
    import nest_asyncio

    nest_asyncio.apply()

# Initialize console for rich output
console = Console()
console.rule("[bold magenta]Agent Evaluation Framework")

# Initialize Weave for experiment tracking
weave.init(WEAVE_PROJECT_NAME)

In [4]:
# Define model configurations to test
model_configs = [
    {"model_id": "google/gemini-1.5-pro", "temperature": 0.2, "name": "Gemini Pro (Low Temp)"},
    {"model_id": "google/gemini-1.5-pro", "temperature": 0.7, "name": "Gemini Pro (High Temp)"},
    {"model_id": "google/gemini-2.0-flash-lite", "temperature": 0.2, "name": "Gemini Flash Lite (Low Temp)"},
    {"model_id": "google/gemini-2.0-flash-lite", "temperature": 0.7, "name": "Gemini Flash Lite (High Temp)"},
    #Any other Gemini or OSS model from vertex
]

# Customer Support Agent Architecture

This section describes the architecture and implementation of our customer support agent system, which is built using a combination of LLM-powered components and specialized tools.

## Agent Overview

The customer support agent is designed to handle a variety of e-commerce related queries by leveraging:

1. **Foundation Models**: The agent can be powered by different models including Gemini (1.5 Pro, 2.0 Flash, etc.) through Vertex AI.

2. **Specialized Tools**: A collection of purpose-built tools that provide domain-specific functionality:
   - `ProductSearchTool`: Searches the product catalog by name, category, or description
   - `OrderStatusTool`: Checks the status of customer orders
   - `CategoryBrowseTool`: Allows browsing products by category
   - `PriceCheckTool`: Retrieves pricing information for specific products
   - `CustomerOrderHistoryTool`: Retrieves order history for customers

3. **Realistic Data**: The system uses realistic product and order data derived from Amazon reviews to provide a realistic customer support experience.

## Technical Implementation

The implementation consists of two main components:

1. **`customer_support_agent.py`**: Contains the tool implementations and agent creation logic. The `create_customer_support_agent()` function configures the agent with the specified model, temperature, planning capabilities, and tools.

2. **`vertex_model.py`**: Provides the model implementations that connect to Vertex AI:
   - `VertexAIServerModel`: Base implementation for connecting to Vertex AI endpoints
   - `WeaveVertexAIServerModel`: Extends the base model with Weights & Biases Weave tracking for experiment monitoring

## Configuration Options

The agent can be configured with various parameters:
- Model selection (Gemini 1.5 Pro, 2.0 Flash, etc.)
- Temperature settings for controlling response randomness
- Planning interval to determine how often the agent should plan its actions
- Maximum steps to limit the complexity of interactions

This architecture enables comprehensive evaluation of different model configurations and parameters to determine the optimal setup for customer support scenarios.


In [ ]:
console.print("[bold blue]Creating customer support agent[/bold blue]")

# Initialize a customer support agent for generating high-quality evaluation data
base_agent = create_customer_support_agent(
    use_weave=True,                    # Enable Weave for experiment tracking
    model_id="google/gemini-2.0-flash", # Use Gemini 2.0 for fast and accurate generation
    temperature=0.1,                   # Low temperature for consistent, deterministic outputs
    planning_interval=2,               # Plan every 2 steps for better reasoning
    max_steps=4                        # Allow up to 4 steps to handle medium-complex queries
)

In [ ]:
base_agent.run("What is the best item in the category of book?")

# Synthetic Dataset Generation for Agent Evaluation

This section explains how we generate realistic test data to evaluate our customer support agent across different configurations and scenarios.

## Why We Need Synthetic Evaluation Data

Testing with synthetic data helps us:

1. **Compare Models Fairly**: We can test different models (Gemini Pro vs Flash) and settings (temperature values) on the exact same customer queries.

2. **Save Time and Resources**: Creating test data is faster and cheaper than collecting real customer conversations.

3. **Cover Edge Cases**: We can include challenging scenarios that might be rare in real data but important to test.

4. **Ensure Consistent Quality**: By filtering examples that meet quality thresholds, we build a reliable benchmark dataset.

## How the Dataset Generator Works

The `DatasetGenerator` class creates evaluation data through several steps:

1. **Creating Realistic Queries**: Generates e-commerce questions using real product IDs, categories, and customer information:
   - "I'm looking for products in the Books category. What do you have?"
   - "Can you check the status of my order OD123456?"
   - "What's your best Electronics product? I need something reliable."

2. **Recording Agent Behavior**: Runs the agent on these queries and captures:
   - Which tools the agent used (ProductSearch, OrderStatus, etc.)
   - The arguments passed to each tool
   - The agent's reasoning at each step
   - The final response to the user

3. **Evaluating Quality**: Uses a judge model to score:
   - Final response quality (0-1 score)
   - Individual step effectiveness (0-1 score per step)
   - Overall trajectory coherence (0-1 score)

4. **Filtering Results**: Only keeps examples that meet quality thresholds (typically 0.7 for each dimension).

## Dataset Structure and Applications

Each example in the final dataset includes:
- The original user prompt
- Expected tool usage sequence
- Validation criteria
- Difficulty rating (easy, medium, hard)
- Metadata about model configuration

This dataset lets us:
1. Determine which model performs best for customer support
2. Find the right temperature settings for different query types
3. Measure whether planning capabilities improve results
4. Identify specific areas where the agent needs improvement

By testing systematically, we can build a more effective customer support agent that handles user queries accurately while using computational resources efficiently.


In [ ]:
# Initialize dataset generator
thresholds={
            "final_response": 0.7,
            "single_step": 0.7,
            "trajectory": 0.7
        }
generator = DatasetGenerator(agent=base_agent, thresholds=thresholds, debug=True)

# Generate comprehensive dataset with different scenarios
console.print("[bold blue]Generating customer support evaluation dataset...[/bold blue]")

dataset = create_customer_support_agent_evaluation_dataset(generator, base_agent, num_prompts=5)  # Adjust number as needed

# Save generated dataset
dataset_path = "customer_support_eval.json"
generator.save_dataset(dataset, dataset_path)

console.print(f"[green]✓[/green] Dataset generation complete! Saved to {dataset_path}")

# Agent Evaluation Framework

This section explains how we evaluate different model configurations using a comprehensive evaluation framework to identify the best-performing agent setup.

## Setting Up the Evaluation Pipeline

The evaluation process begins with initializing our evaluation framework defined in `evaluator.py`.
The below code loads our previously generated synthetic dataset and formats it for evaluation with Vertex AI.

## How the Evaluation Process Works

The `AgentEvaluator` class provides a systematic approach to testing agent performance across multiple dimensions:

1. **Multi-dimensional Metrics**: We evaluate the agent on several key aspects:
   - **Tool Selection Accuracy**: How well the agent chooses appropriate tools
   - **Reasoning Quality**: The logical coherence of the agent's thinking process
   - **Response Correctness**: Accuracy and completeness of final answers
   - **Trajectory Match**: How well the agent's path aligns with expected solutions
   - **Coherence**: Overall clarity and consistency of responses

2. **Vertex AI Integration**: The evaluator uses Vertex AI's evaluation capabilities to score agent responses objectively, reducing human bias in the assessment process.

3. **Weights & Biases Weave Integration**: Results are automatically logged to Weave, enabling:
   - Interactive visualization of agent performance
   - Comparison between different model configurations
   - Tracking of experiments over time
   - Sharing results with team members

4. **Visualization Tools**: The framework generates charts and tables to help identify patterns:
   - Score distribution plots showing performance across metrics
   - Difficulty heatmaps revealing how the agent handles easy vs. hard queries
   - Correlation analysis between trajectory quality and response accuracy

## Practical Applications

This evaluation framework helps us answer key questions:

1. **Which model performs best?** Compare Gemini Pro vs. Flash models on the same test cases.

2. **What temperature setting works better?** Test whether low temperature (0.2) or high temperature (0.7) produces better results.

3. **Where are the weaknesses?** Identify specific query types or metrics where the agent underperforms.

4. **Is planning helpful?** Measure whether enabling planning capabilities improves overall performance.

By running all model configurations through this standardized evaluation process, we can make data-driven decisions about which agent setup to deploy for customer support scenarios, balancing performance and efficiency.


In [ ]:
# Initialize evaluator
console.print("[bold blue]Initializing evaluator...[/bold blue]")
evaluator = AgentEvaluator(verbosity=1, project=os.getenv("VERTEX_PROJECT_ID"), location=os.getenv("VERTEX_LOCATION"))
console.print(f"[green]✓[/green] Evaluator initialized")

all_examples = load_dataset("customer_support_eval.json")
console.print(f"[bold blue]Formatting dataset with {len(all_examples)} examples for evaluation...[/bold blue]")
eval_dataset = evaluator.format_dataset_for_eval(all_examples)
console.print(f"[green]✓[/green] Dataset formatted successfully")

# Running Model Evaluations

This section shows how we test different model configurations against our evaluation dataset to find the best setup for customer support.

## Testing Multiple Configurations

We evaluate four different configurations:
- Gemini Pro with low temperature (0.2)
- Gemini Pro with high temperature (0.7)
- Gemini Flash Lite with low temperature (0.2)
- Gemini Flash Lite with high temperature (0.7)

Each configuration is tested on the same set of customer queries, ensuring a fair comparison.

In [ ]:
@weave.op()
def evaluate_model_config(config: Dict[str, Any], eval_dataset: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Evaluate a specific model configuration and return results"""
    console.print(f"\n[bold blue]Evaluating {config['name']}...[/bold blue]")
    
    # Create agent with this configuration
    agent = create_customer_support_agent(
        use_weave=True,
        model_id=config["model_id"],
        temperature=config["temperature"],
        planning_interval=config.get("planning_interval", 1),
        max_steps=config.get("max_steps", 5)
    )
    
    # Run evaluation with the agent object directly
    results = evaluator.run_evaluation(
        agent=agent, 
        eval_dataset=eval_dataset, 
        output_dir=f"evaluation_results/{config['name'].replace(' ', '_').lower()}",
        weave_project=WEAVE_PROJECT_NAME
    )
    
    # Add configuration details to results
    results["config"] = config
    
    return results

# Run evaluations for all configurations
all_results = []
for config in model_configs:
    results = evaluate_model_config(config, eval_dataset)
    all_results.append(results)
    
    # Display summary results
    console.print(f"\n[bold green]Results for {config['name']}:[/bold green]")
    if "summary_metrics" in results and results["summary_metrics"]:
        table = evaluator._render_summary_table(results["summary_metrics"])
        console.print(table)
    else:
        console.print("[yellow]No summary metrics available[/yellow]")

# Navigate to Weave to compare results!!

Go to the project set in `WEAVE_PROJECT_NAME` in `config.py`

# Analyzing Model Performance Patterns

After running evaluations on different model configurations, we analyze the results to uncover key patterns and insights.

## Response Quality by Model Type

We compare how Gemini Pro and Gemini Flash Lite perform on response quality metrics. The analysis reveals which model provides more accurate and helpful customer support responses across our test cases.

## Impact of Temperature Settings

By examining how temperature affects performance, we can determine whether lower temperatures (0.2) produce more consistent, reliable responses or if higher temperatures (0.7) generate more helpful, creative solutions for customer queries.

## Performance Across Query Difficulty

We analyze how different models handle queries of varying complexity:
- Which model excels at simple, straightforward questions?
- Which configuration best handles complex, multi-part customer issues?
- Are there specific difficulty levels where one model significantly outperforms others?


In [ ]:
# Compare models and their impact on response quality
import numpy as np
import re

# Create a function to extract the base model name (ignoring parentheses)
def get_base_model_name(full_name):
    # Remove anything in parentheses and trim whitespace
    base_name = re.sub(r'\s*\([^)]*\)', '', full_name).strip()
    return base_name

# Prepare response data
response_data = []

for result in all_results:
    config = result["config"]
    detailed_metrics = result["detailed_metrics"]
    
    # Get base model name
    base_model = get_base_model_name(config["name"])
    
    # Find response quality column
    response_col = next((col for col in detailed_metrics.columns if "response" in col.lower() and "score" in col.lower()), None)
    
    if response_col and not detailed_metrics.empty:
        # Add model info to each row
        model_data = detailed_metrics.copy()
        model_data["Base Model"] = base_model
        model_data["Temperature"] = config["temperature"]
        
        # Select only needed columns
        model_data = model_data[["Base Model", "Temperature", response_col]].rename(columns={response_col: "Response Quality"})
        response_data.append(model_data)

# Combine all data
if response_data:
    combined_data = pd.concat(response_data, ignore_index=True)
    
    # Create a simple, clean figure
    plt.figure(figsize=(10, 6))
    
    # Create a basic boxplot
    ax = sns.boxplot(x="Base Model", y="Response Quality", data=combined_data, 
                    palette="pastel", width=0.6)
    
    # Simple, clean styling
    plt.title("Response Quality by Model Type", fontsize=16)
    plt.xlabel("Model", fontsize=14)
    plt.ylabel("Response Quality Score", fontsize=14)
    
    # Add the overall average line
    overall_mean = combined_data["Response Quality"].mean()
    plt.axhline(y=overall_mean, color='red', linestyle='--', linewidth=1.5)
    plt.text(len(combined_data["Base Model"].unique()) - 0.5, overall_mean + 0.05, 
             f"Overall Mean: {overall_mean:.2f}", 
             ha='right', color='red', fontweight='bold')
    
    # Add mean values as annotations - now below the boxes
    for i, model in enumerate(combined_data["Base Model"].unique()):
        model_data = combined_data[combined_data["Base Model"] == model]
        mean_score = model_data["Response Quality"].mean()
        
        # Calculate the bottom of the box
        q1 = model_data["Response Quality"].quantile(0.25)
        q3 = model_data["Response Quality"].quantile(0.75)
        iqr = q3 - q1
        bottom = max(model_data["Response Quality"].min(), q1 - 1.5 * iqr)
        
        # Place text below the box
        plt.text(i, bottom - 0.2, f"Mean: {mean_score:.2f}", 
                ha='center', va='top', fontweight='bold', fontsize=11)
    
    # Save and show
    plt.tight_layout()
    plt.savefig("model_response_quality.png", dpi=300)
    plt.show()
    
    # Create a comprehensive summary table with all metrics by model
    console.rule("[bold magenta]Model Performance Summary")
    
    # Get all metrics from all results
    all_metrics = {}
    for result in all_results:
        config = result["config"]
        metrics = result["summary_metrics"]
        base_model = get_base_model_name(config["name"])
        
        if base_model not in all_metrics:
            all_metrics[base_model] = {}
        
        # Add all mean metrics
        for metric_key, value in metrics.items():
            if "/Mean" in metric_key:
                # Clean up metric name for display
                clean_metric = metric_key.replace("/Mean", "").replace("_score", "").replace("_", " ").title()
                
                if clean_metric not in all_metrics[base_model]:
                    all_metrics[base_model][clean_metric] = []
                
                all_metrics[base_model][clean_metric].append(value)
    
    # Create a table with all metrics
    summary_table = Table(title="All Metrics by Model Type")
    
    # Add model column
    summary_table.add_column("Model", style="cyan")
    
    # Find all unique metrics
    all_metric_names = set()
    for model_metrics in all_metrics.values():
        all_metric_names.update(model_metrics.keys())
    
    # Add columns for each metric
    for metric in sorted(all_metric_names):
        summary_table.add_column(metric, style="green")
    
    # Add rows for each model
    for model, metrics in all_metrics.items():
        row_values = [model]
        
        for metric in sorted(all_metric_names):
            if metric in metrics:
                # Calculate average if there are multiple values (from different temperatures)
                avg_value = sum(metrics[metric]) / len(metrics[metric])
                row_values.append(f"{avg_value:.2f}")
            else:
                row_values.append("N/A")
        
        summary_table.add_row(*row_values)
    
    console.print(summary_table)
else:
    console.print("[yellow]Warning: No response quality data available for visualization[/yellow]")

In [ ]:
# Analyze performance by query difficulty
import numpy as np
import re

# Create a function to extract the base model name (ignoring parentheses)
def get_base_model_name(full_name):
    # Remove anything in parentheses and trim whitespace
    base_name = re.sub(r'\s*\([^)]*\)', '', full_name).strip()
    return base_name

# Create a dataframe with all detailed metrics and model info
all_detailed_metrics = []

for result in all_results:
    config = result["config"]
    detailed_metrics = result["detailed_metrics"].copy()
    
    # Skip if detailed_metrics is empty
    if detailed_metrics.empty:
        continue
    
    # Get base model name and temperature
    base_model = get_base_model_name(config["name"])
    temperature = config["temperature"]
    
    # Add model info to detailed metrics
    detailed_metrics["Base Model"] = base_model
    detailed_metrics["Temperature"] = temperature
    detailed_metrics["Model Config"] = f"{base_model} (T={temperature})"
    
    all_detailed_metrics.append(detailed_metrics)

# Only proceed if we have data
if all_detailed_metrics:
    all_metrics_df = pd.concat(all_detailed_metrics, ignore_index=True)
    
    # Check if difficulty column exists
    difficulty_col = next((col for col in all_metrics_df.columns if col.lower() == "difficulty"), None)
    
    # Find response correctness column
    response_col = next((col for col in all_metrics_df.columns if "response" in col.lower() and "score" in col.lower()), None)
    
    if difficulty_col and response_col:
        # Convert difficulty to string to ensure proper categorical plotting
        all_metrics_df[difficulty_col] = all_metrics_df[difficulty_col].astype(str)
        
        # Create a simple, clean figure
        plt.figure(figsize=(12, 7))
        
        # Plot performance by difficulty
        ax = sns.boxplot(x=difficulty_col, y=response_col, data=all_metrics_df, 
                        palette="pastel", width=0.6)
        
        # Simple, clean styling
        plt.title("Response Quality by Query Difficulty", fontsize=16)
        plt.xlabel("Query Difficulty", fontsize=14)
        plt.ylabel("Response Quality Score", fontsize=14)
        
        # Add the overall average line
        overall_mean = all_metrics_df[response_col].mean()
        plt.axhline(y=overall_mean, color='red', linestyle='--', linewidth=1.5)
        plt.text(len(all_metrics_df[difficulty_col].unique()) - 0.5, overall_mean + 0.05, 
                f"Overall Mean: {overall_mean:.2f}", 
                ha='right', color='red', fontweight='bold')
        
        # Add mean values as annotations below the boxes
        for i, difficulty in enumerate(sorted(all_metrics_df[difficulty_col].unique())):
            difficulty_data = all_metrics_df[all_metrics_df[difficulty_col] == difficulty]
            mean_score = difficulty_data[response_col].mean()
            
            # Calculate the bottom of the box
            q1 = difficulty_data[response_col].quantile(0.25)
            q3 = difficulty_data[response_col].quantile(0.75)
            iqr = q3 - q1
            bottom = max(difficulty_data[response_col].min(), q1 - 1.5 * iqr)
            
            # Place text below the box
            plt.text(i, bottom - 0.2, f"Mean: {mean_score:.2f}", 
                    ha='center', va='top', fontweight='bold', fontsize=11)
        
        # Save and show
        plt.tight_layout()
        plt.savefig("response_quality_by_difficulty.png", dpi=300)
        plt.show()
        
        # Create a detailed table showing best model for each difficulty level
        console.rule("[bold magenta]Best Model Configuration by Difficulty")
        
        detailed_table = Table(title="Best Model Configuration for Each Difficulty Level")
        detailed_table.add_column("Difficulty", style="cyan")
        detailed_table.add_column("Best Model", style="green")
        detailed_table.add_column("Temperature", style="yellow")
        detailed_table.add_column("Score", style="magenta")
        detailed_table.add_column("Improvement Over Avg", style="blue")
        
        # Add rows for each difficulty level
        for difficulty in sorted(all_metrics_df[difficulty_col].unique()):
            difficulty_data = all_metrics_df[all_metrics_df[difficulty_col] == difficulty]
            avg_score = difficulty_data[response_col].mean()
            
            # Group by model and temperature to find the best configuration
            grouped = difficulty_data.groupby(["Base Model", "Temperature"])[response_col].mean().reset_index()
            best_idx = grouped[response_col].idxmax()
            best_model = grouped.loc[best_idx, "Base Model"]
            best_temp = grouped.loc[best_idx, "Temperature"]
            best_score = grouped.loc[best_idx, response_col]
            
            # Calculate improvement over average
            improvement = best_score - avg_score
            improvement_pct = (improvement / avg_score) * 100 if avg_score > 0 else 0
            
            detailed_table.add_row(
                difficulty,
                best_model,
                f"{best_temp}",
                f"{best_score:.2f}",
                f"+{improvement:.2f} ({improvement_pct:.1f}%)"
            )
        
        console.print(detailed_table)
        
        # Create a table showing all metrics by difficulty
        console.rule("[bold magenta]All Metrics by Difficulty")
        
        # Find all score columns
        score_columns = [col for col in all_metrics_df.columns 
                        if "score" in col.lower() and col != response_col]
        
        # Add response column to the list if not already included
        if response_col and response_col not in score_columns:
            score_columns.append(response_col)
        
        # Create the table
        metrics_by_difficulty_table = Table(title="All Metrics by Difficulty Level")
        metrics_by_difficulty_table.add_column("Difficulty", style="cyan")
        
        # Add columns for each metric
        for col in sorted(score_columns):
            # Clean up column name for display
            clean_name = col.replace("_score", "").replace("_", " ").title()
            metrics_by_difficulty_table.add_column(clean_name, style="green")
        
        # Add rows for each difficulty level
        for difficulty in sorted(all_metrics_df[difficulty_col].unique()):
            difficulty_data = all_metrics_df[all_metrics_df[difficulty_col] == difficulty]
            
            row_values = [difficulty]
            
            for col in sorted(score_columns):
                if col in difficulty_data.columns:
                    avg_score = difficulty_data[col].mean()
                    row_values.append(f"{avg_score:.2f}")
                else:
                    row_values.append("N/A")
            
            metrics_by_difficulty_table.add_row(*row_values)
        
        console.print(metrics_by_difficulty_table)
    else:
        console.print("[yellow]Warning: Missing difficulty or response correctness columns in metrics data[/yellow]")
else:
    console.print("[yellow]Warning: No detailed metrics available for difficulty analysis[/yellow]")

In [ ]:
# Analyze performance by temperature
import numpy as np
import re

# Create a function to extract the base model name (ignoring parentheses)
def get_base_model_name(full_name):
    # Remove anything in parentheses and trim whitespace
    base_name = re.sub(r'\s*\([^)]*\)', '', full_name).strip()
    return base_name

# Create a dataframe with all detailed metrics and model info
all_detailed_metrics = []

for result in all_results:
    config = result["config"]
    detailed_metrics = result["detailed_metrics"].copy()
    
    # Skip if detailed_metrics is empty
    if detailed_metrics.empty:
        continue
    
    # Get base model name and temperature
    base_model = get_base_model_name(config["name"])
    temperature = config["temperature"]
    
    # Add model info to detailed metrics
    detailed_metrics["Base Model"] = base_model
    detailed_metrics["Temperature"] = temperature
    detailed_metrics["Model Config"] = f"{base_model} (T={temperature})"
    
    all_detailed_metrics.append(detailed_metrics)

# Only proceed if we have data
if all_detailed_metrics:
    all_metrics_df = pd.concat(all_detailed_metrics, ignore_index=True)
    
    # Find response correctness column
    response_col = next((col for col in all_metrics_df.columns if "response" in col.lower() and "score" in col.lower()), None)
    
    if "Temperature" in all_metrics_df.columns and response_col:
        # Convert temperature to string to ensure proper categorical plotting
        all_metrics_df["Temperature"] = all_metrics_df["Temperature"].astype(str)
        
        # Create a simple, clean figure
        plt.figure(figsize=(12, 7))
        
        # Plot performance by temperature
        ax = sns.boxplot(x="Temperature", y=response_col, data=all_metrics_df, 
                        palette="pastel", width=0.6)
        
        # Simple, clean styling
        plt.title("Response Quality by Temperature", fontsize=16)
        plt.xlabel("Temperature", fontsize=14)
        plt.ylabel("Response Quality Score", fontsize=14)
        
        # Add the overall average line
        overall_mean = all_metrics_df[response_col].mean()
        plt.axhline(y=overall_mean, color='red', linestyle='--', linewidth=1.5)
        plt.text(len(all_metrics_df["Temperature"].unique()) - 0.5, overall_mean + 0.05, 
                f"Overall Mean: {overall_mean:.2f}", 
                ha='right', color='red', fontweight='bold')
        
        # Add mean values as annotations below the boxes
        for i, temp in enumerate(sorted(all_metrics_df["Temperature"].unique())):
            temp_data = all_metrics_df[all_metrics_df["Temperature"] == temp]
            mean_score = temp_data[response_col].mean()
            
            # Calculate the bottom of the box
            q1 = temp_data[response_col].quantile(0.25)
            q3 = temp_data[response_col].quantile(0.75)
            iqr = q3 - q1
            bottom = max(temp_data[response_col].min(), q1 - 1.5 * iqr)
            
            # Place text below the box
            plt.text(i, bottom - 0.2, f"Mean: {mean_score:.2f}", 
                    ha='center', va='top', fontweight='bold', fontsize=11)
        
        # Save and show
        plt.tight_layout()
        plt.savefig("response_quality_by_temperature.png", dpi=300)
        plt.show()
        
        # Create a detailed table showing best model for each temperature
        console.rule("[bold magenta]Best Model Configuration by Temperature")
        
        detailed_table = Table(title="Best Model Configuration for Each Temperature")
        detailed_table.add_column("Temperature", style="cyan")
        detailed_table.add_column("Best Model", style="green")
        detailed_table.add_column("Score", style="magenta")
        detailed_table.add_column("Improvement Over Avg", style="blue")
        
        # Add rows for each temperature level
        for temp in sorted(all_metrics_df["Temperature"].unique()):
            temp_data = all_metrics_df[all_metrics_df["Temperature"] == temp]
            avg_score = temp_data[response_col].mean()
            
            # Group by model to find the best configuration
            grouped = temp_data.groupby(["Base Model"])[response_col].mean().reset_index()
            best_idx = grouped[response_col].idxmax()
            best_model = grouped.loc[best_idx, "Base Model"]
            best_score = grouped.loc[best_idx, response_col]
            
            # Calculate improvement over average
            improvement = best_score - avg_score
            improvement_pct = (improvement / avg_score) * 100 if avg_score > 0 else 0
            
            detailed_table.add_row(
                temp,
                best_model,
                f"{best_score:.2f}",
                f"+{improvement:.2f} ({improvement_pct:.1f}%)"
            )
        
        console.print(detailed_table)
        
        # Create a table showing all metrics by temperature
        console.rule("[bold magenta]All Metrics by Temperature")
        
        # Find all score columns
        score_columns = [col for col in all_metrics_df.columns 
                        if "score" in col.lower() and col != response_col]
        
        # Add response column to the list if not already included
        if response_col and response_col not in score_columns:
            score_columns.append(response_col)
        
        # Create the table
        metrics_by_temp_table = Table(title="All Metrics by Temperature")
        metrics_by_temp_table.add_column("Temperature", style="cyan")
        
        # Add columns for each metric
        for col in sorted(score_columns):
            # Clean up column name for display
            clean_name = col.replace("_score", "").replace("_", " ").title()
            metrics_by_temp_table.add_column(clean_name, style="green")
        
        # Add rows for each temperature level
        for temp in sorted(all_metrics_df["Temperature"].unique()):
            temp_data = all_metrics_df[all_metrics_df["Temperature"] == temp]
            
            row_values = [temp]
            
            for col in sorted(score_columns):
                if col in temp_data.columns:
                    avg_score = temp_data[col].mean()
                    row_values.append(f"{avg_score:.2f}")
                else:
                    row_values.append("N/A")
            
            metrics_by_temp_table.add_row(*row_values)
        
        console.print(metrics_by_temp_table)
    else:
        console.print("[yellow]Warning: Missing temperature or response correctness columns in metrics data[/yellow]")
else:
    console.print("[yellow]Warning: No detailed metrics available for temperature analysis[/yellow]")

# Conclusion

Our systematic evaluation of different model configurations provides a framework for making data-driven decisions about customer support agent implementation.

## Next Steps

With the evaluation results in hand, we can now:

1. **Select the optimal configuration** based on actual performance metrics rather than assumptions or theoretical capabilities.

2. **Understand trade-offs** between different models (Gemini Pro vs. Flash Lite) and temperature settings (0.2 vs. 0.7) for our specific customer support scenarios.

3. **Identify improvement areas** by focusing on the metrics where even our best configuration underperformed.

4. **Expand our evaluation dataset** to include more diverse customer queries and edge cases.

This evaluation framework allows us to continually refine our customer support agent as new models become available or as customer needs evolve. By measuring performance objectively across multiple dimensions, we can ensure we're deploying the most effective solution for our specific use case.


In [ ]:
console.rule("[bold magenta]Evaluation Summary")

# First create the comparison dataframe if it doesn't exist
comparison_data = []
for result in all_results:
    config = result["config"]
    metrics = result["summary_metrics"]
    
    row = {
        "Model": config["name"],
        "Model ID": config["model_id"],
        "Temperature": config["temperature"]
    }
    
    # Add only mean values of metrics for cleaner comparison
    for metric_key, value in metrics.items():
        if "/Mean" in metric_key:
            # Clean up metric name for display
            clean_metric = metric_key.replace("/Mean", "").replace("_score", "").replace("_", " ").title()
            row[clean_metric] = value
    
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)

# Get metric columns (excluding metadata columns)
metric_columns = [col for col in comparison_df.columns 
                 if col not in ["Model", "Model ID", "Temperature"]]

# Calculate best model for each metric
best_models = {}
worst_models = {}
for metric in metric_columns:
    best_idx = comparison_df[metric].idxmax()
    worst_idx = comparison_df[metric].idxmin()
    best_models[metric] = {
        "model": comparison_df.loc[best_idx, "Model"],
        "score": comparison_df.loc[best_idx, metric],
        "temperature": comparison_df.loc[best_idx, "Temperature"]
    }
    worst_models[metric] = {
        "model": comparison_df.loc[worst_idx, "Model"],
        "score": comparison_df.loc[worst_idx, metric]
    }

# Create summary table
summary_table = Table(title="Evaluation Summary")
summary_table.add_column("Metric", style="cyan")
summary_table.add_column("Best Model", style="green")
summary_table.add_column("Score", style="yellow")
summary_table.add_column("Temperature", style="magenta")

for metric, data in best_models.items():
    summary_table.add_row(
        metric, 
        data["model"], 
        f"{data['score']:.2f}",
        f"{data['temperature']}"
    )

console.print(summary_table)

# Calculate average scores by model
model_avg_scores = {}
for _, row in comparison_df.iterrows():
    model = row["Model"]
    scores = [row[m] for m in metric_columns]
    avg_score = sum(scores) / len(scores)
    model_avg_scores[model] = avg_score

# Find overall best model
overall_best_model = max(model_avg_scores.items(), key=lambda x: x[1])

# Find best temperature if we have temperature data
temp_analysis = {}
if "Temperature" in comparison_df.columns:
    for temp in comparison_df["Temperature"].unique():
        temp_df = comparison_df[comparison_df["Temperature"] == temp]
        if not temp_df.empty:
            avg_scores = [temp_df[m].mean() for m in metric_columns]
            temp_analysis[temp] = sum(avg_scores) / len(avg_scores)
    
    if temp_analysis:  # Check if we have any temperature data
        best_temp = max(temp_analysis.items(), key=lambda x: x[1])
        worst_temp = min(temp_analysis.items(), key=lambda x: x[1])

# Find strongest and weakest metrics
metric_avgs = {metric: comparison_df[metric].mean() for metric in metric_columns}
strongest_metric = max(metric_avgs.items(), key=lambda x: x[1])
weakest_metric = min(metric_avgs.items(), key=lambda x: x[1])

# Generate dynamic key findings
key_findings = f"""
[bold]Key Findings:[/bold]

1. [bold cyan]Overall Performance:[/bold cyan] {overall_best_model[0]} is the best performing model with an average score of {overall_best_model[1]:.2f} across all metrics.

2. [bold cyan]Metric-Specific Performance:[/bold cyan]
   - Best metric: {strongest_metric[0]} (avg: {strongest_metric[1]:.2f})
   - Most challenging metric: {weakest_metric[0]} (avg: {weakest_metric[1]:.2f})
"""

# Add temperature findings if available
if "Temperature" in comparison_df.columns and temp_analysis:
    key_findings += f"""
3. [bold cyan]Temperature Impact:[/bold cyan] Temperature {best_temp[0]} yielded the best overall performance (avg: {best_temp[1]:.2f}), while temperature {worst_temp[0]} performed worst (avg: {worst_temp[1]:.2f}).
"""

# Add model-specific insights
key_findings += """
4. [bold cyan]Model-Specific Insights:[/bold cyan]
"""

for metric in metric_columns:
    best = best_models[metric]
    worst = worst_models[metric]
    key_findings += f"   - {metric}: {best['model']} excels ({best['score']:.2f}), while {worst['model']} struggles ({worst['score']:.2f}).\n"

# Add next steps
key_findings += """
[bold]Next Steps:[/bold]

1. Deploy the best-performing model configuration ({}) for production use
2. Focus on improving performance in the weakest metric area: {}
3. {}
4. Expand the evaluation dataset with more diverse scenarios
5. Implement continuous evaluation to monitor agent performance over time
""".format(
    overall_best_model[0],
    weakest_metric[0],
    f"Conduct further temperature tuning around the optimal value of {best_temp[0]}" if "Temperature" in comparison_df.columns and temp_analysis else "Experiment with different temperature settings to optimize performance"
)

console.print(key_findings)